In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 3, Finished, Available)

In [5]:
%%sql
create table if not exists lh_ecormece.gold_Dimshipmode
(
    ShipMode_ID	Long	,
    Ship_Mode	string	,
    Created_TS	timestamp,
    Modified_TS timestamp
)
using delta

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
max_date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from lh_ecormece.gold_Dimshipmode").first()[0]

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 8, Finished, Available)

In [7]:
max_date

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 9, Finished, Available)

'1900-01-01'

In [10]:
df_bronze = spark.sql("select DISTINCT Ship_Mode from lh_ecormece.bronze_sales WHERE Modified_TS > '{}'".format(max_date))

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 12, Finished, Available)

In [12]:
display(df_bronze.head(10))

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8ba2affd-3d45-44cd-8cdf-593192a47dc9)

In [15]:
max_ID = spark.sql("SELECT coalesce(max(ShipMode_ID),0) from lh_ecormece.gold_dimshipmode").first()[0]

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 17, Finished, Available)

In [16]:
max_ID

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 18, Finished, Available)

0

In [17]:
df_final = df_bronze.withColumn("ShipMode_ID",monotonically_increasing_id()+max_ID+1)

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 19, Finished, Available)

In [20]:
df_final.createOrReplaceTempView('View_ShipMode')

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 22, Finished, Available)

In [21]:
%%sql
MERGE INTO lh_ecormece.gold_dimshipmode as sm 
using View_ShipMode as vs
on sm.Ship_Mode=vs.Ship_Mode
when MATCHED then
    UPDATE set
         Modified_TS = CURRENT_TIMESTAMP()
when not MATCHED THEN
    INSERT 
    (
        ShipMode_ID,
        Ship_Mode,
        Created_TS,
        Modified_TS
    )
    VALUES
    (
        vs.ShipMode_ID,
        vs.Ship_Mode,
        CURRENT_TIMESTAMP(),
        CURRENT_TIMESTAMP()
    )

StatementMeta(, 94efa436-3bfd-4f73-a516-cfa433cd8f12, 23, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>